# Testdaten und Confusion Matrix

In diesem Teil beschäftigst du dich damit, wie du die Abstraktionsfähigkeit eines Modells und damit seine Vorhersagequalität messen kannst.

Bisher hast du "nur" überprüft, wie gut das Modell das gelernte *reproduzieren konnte*. Jetzt wirst du echte Vorhersagen mit unbekannten Daten durchführen.

## Daten einladen

Wie gewohnt lädst du die linguistisch analysierten Daten ein:

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM nlp_articles WHERE datePublished<'2021-01-01' ORDER BY datePublished", 
                 sql, index_col="id", parse_dates=["datePublished"])

In diesem Teil betrachtest du nur die Klassifikation nach Autoren. Genauso kannst du das natürlich für Keywords (einzeln!) oder für die Kommentare durchführen:

## Daten für Autoren-Klassifikation vorbereiten

Du bestimmst zunächst die Top-Autoren:

In [ ]:
top_authors = df.groupby("author").count().sort_values("title", ascending=False).head(20)[["title"]]

Alle Autoren sollen gleich viele Artikel erhalten:

In [ ]:
min_articles = min(top_authors["title"])
adf = pd.concat([df[df["author"] == author].sample(min_articles, random_state=42)
                     for author in top_authors.index.values])

Nun kannst du die Daten vektorisieren:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.de.stop_words import STOP_WORDS as stop_words
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words, min_df=2)
tfidf_vectors = tfidf_vectorizer.fit_transform(adf["nav"])
tfidf_vectors

Nun möchtest du das Datenset in zwei voneinander unabhängige teilen:
1. Die *Trainingsdaten* dienen dem Modell zum Training. Mit diesen Daten soll es lernen und seine Abstraktionsfähigkeit erlangen.
2. Die *Testdaten*, mit denen du das gelernte verifizierst.

Wichtig: Sowohl die Trainings- als auch die Testdaten müssen schon vorgelabelt sein, d.h. du musst das richtige Ergebnis kennen.

Den Split in Trainings- und Testdaten kannst du mit der Funktion `train_test_split` von `scikit-learn` durchführen. Dieser Funktion übergibst du die Vektoren und die Ergebnisse. Außerdem kannst du noch angeben, wie viele Daten du im Trainingsset haben willst (hier 75%). Der `random_state` dient der Reproduzierbarkeit, das kennst du schon.

Eine Schwierigkeit kommt jetzt noch dazu: du musst darauf achten, dass von jedem Autor der gleiche Anteil in den Trainings- und Testdaten liegt, sonst ist das Training wenig ausgewogen. Dazu nutzt du den `stratify`-Parameter, um das Datenset fair ([stratifiziert](https://de.wikipedia.org/wiki/Geschichtete_Zufallsstichprobe)) aufzubauen:

In [ ]:
from sklearn.model_selection import train_test_split
(X_train, X_test, y_train, y_test) = train_test_split(tfidf_vectors, adf["author"].values, 
                                                      train_size=0.75, random_state=42,
                                                      stratify=adf["author"].values)

## Training und Auswertung SVM

Das Training läuft nun (annhähernd) so, wie du es schon kennst. Es gibt einen kleinen Unterschied, du trainierst das Modell nur mit den Trainingsdaten:

In [ ]:
from sklearn.svm import SVC
svm = SVC()
svc = svm.fit(X_train, y_train)

Jetzt kannst du sowohl die Ergebnisse für die Trainings- als auch für die Testdaten vorhersagen:

In [ ]:
pred_train = svc.predict(X_train)
pred_test  = svc.predict(X_test)

Nun wird es spannend! Wie hat sich das Modell in der Klassifikation der Trainingsdaten geschlagen?

In [ ]:
import numpy as np
np.unique(pred_train == y_train, return_counts=True)

Das hat wie erwartet sehr gut funktioniert mit nur sechs Fehlern.

Schau dir nun die Ergebnisse bei den Testdaten an:

In [ ]:
np.unique(pred_test == y_test, return_counts=True)

Das ist ein erheblicher Unterschied und deutet auf ein auswendig gelerntes Modell hin (sog. (Overfitting)[https://de.wikipedia.org/wiki/%C3%9Cberanpassung]).

Damit ist die echte Vorhersagekraft des Modells leider nicht so gut, wie du (vielleicht) ursprünglich gedacht hast.

## Verteilung der Fehler

Wie verteilen sich die Fehler in den klassifizierten Daten des Datensets? Eigentlich müssten alle Autoren gleich häufig vorkommen. Das kannst du relativ einfach ermitteln:

In [ ]:
pt = pd.DataFrame(pred_test, columns=["author"])
pt["count"] = 1
pt.groupby("author").count().plot.barh()

## Confusion Matrix

Das ist nicht *fair*, `Tilman Wittenhorst` hat bei weitem zu viele Artikel zugeordnet bekommen!

Um herauszufinden, wie sich die Fehler verteilen, kannst du die sog. [Confusion Matrix](https://de.wikipedia.org/wiki/Beurteilung_eines_bin%C3%A4ren_Klassifikators#Wahrheitsmatrix:_Richtige_und_falsche_Klassifikationen) berechnen:

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, pred_test, labels=top_authors.index.values)

In den Zeilen stehen die echten Ergebnisse, in den Spalten die vorhergesagten. Einen Treffer findest du also auf der Diagonalen.

Das ist schön, aber leider *sehr unübersichtlich*. Mithilfe von `seaborn` kannst du das viel besser in einer Heatmap darstellen:

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(10,8))
sns.heatmap(confusion_matrix(y_test, pred_test, labels=top_authors.index.values), 
            xticklabels=top_authors.index.values, yticklabels=top_authors.index.values)

Du siehst, dass `Tilman Wittenhorst` sehr oft vorhergesagt wurde, aber leider auch sehr häufig falsch. Das würde auch deiner Erwartung entsprechen, die du nach dem Balkendiagramm oben hast.

Mit anderen Klassifikatoren erzielst du andere, möglicherweise auch bessere Ergebnisse. Das kannst du einfach durch den Austausch der Klassifikatoren oben erreichen (aus dem letzten Notebook).

## Confusion Matrix vermittelt Einsicht

Anhand der Confusion Matrix siehst du gut, wo der Klassifikator *daneben liegt*. Allerdings eignet sich die Matrix noch nicht gut als Maß für die Performance, weil viel zu viele Einzelwerte vorhanden sind.

Eine Lösung dafür lernst du in den nächsten Kapiteln kennen.